# Big Data Final Project

## Team: Tri Dang (814009034), Hsuan Yu Liu (823327369)

## Goal

Predict the rating of a business based on its reviews.

## Process

1. Load the data
2. Clean the data
3. Train the model and predict dataset without using PCA
4. Train the model and predict dataset using PCA
5. Analyze and conclude the results

## To Run

1. Install PyEnchant using pip
2. Run the notebook

## 1. Load the data

In [1]:
filename = 'review.json'
outputname = 'smaller_review.json'
output_folder = "output"
download_name = 'clean_review.csv'

In [2]:
import os
import json
import re
import pandas as pd
from matplotlib import pyplot as plt
import nltk
from nltk.stem.lancaster import LancasterStemmer
from collections import Counter
import time
import enchant
import numpy as np

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession

import sklearn
from sklearn.cluster import KMeans
from sklearn import decomposition
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

In [3]:
program_time = time.time()

dictionary = enchant.Dict("en_US")
st = LancasterStemmer()
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/tridang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# Configuration

# what size we want to start with
initial_data_size = 10000
# size we want to read into Pandas DataFrame. # of Chunks = initial_data_size / chunk_size
chunk_size = 100
# minimum of words we want in a review 
min_keep_size = 80
# how many features we want to keep
pca_components = 1000

### Reducing data size

Initially, we wanted to use PySpark to clean our data, since the data contained over 6 million rows. We spent an entire day trying to load the data into PySpark DataFrame and perform row transformations to clean our data. However, at the end of the day, we realized that it was a complicated task that could be completed easier in Pandas DataFrame. We were much more familiar with Pandas, and decided that as a team, it was better for us to use Pandas, than to spend much time learning PySpark DataFrame. This would allow us to reduce our code development time. We also understand that using Pandas would also increase our program run time, since it is not suited for large amounts of data. However, it was a tradeoff that we were okay with, since we wanted to be able to quickly develop and test our program. This means that we are only using a small subset of our real dataset, only 10,000 out of 6 million.

In [ ]:
spark = SparkSession.builder \
    .appName("Yelp") \
    .getOrCreate()

In [ ]:
# get a smaller subset of the data to use for development
yelp_df = spark.read.json(filename)
yelp_df = yelp_df.limit(initial_data_size)

fewer_parts = yelp_df.coalesce(1)
fewer_parts.write.format('json').save(output_folder)

      
for filename in os.listdir(output_folder): 
    src = output_folder + "/" + filename
    file, file_extension = os.path.splitext(src)
    if file_extension == '.json':
        os.rename(src, outputname) 

## 2. Cleaning the Data

Before we were able to clean the data, we had to load it into Pandas DataFrame. Due to the size of our dataset, the only way we were able to load it efficiently was to load it in chunks. We tested chunksize of 1000 and 100 and found that the smaller chunksize reduced our processing time by 2/3. 

For the cleaning, we have to take out uncessary characters such as: integers, numerics, single letters (except 'a' and 'i'), quotes, 'markups' (\n, \r, etc.), and other symbols. This will leave us with only characters a-z and spaces. After that, we parse the Review Text into tokens, using the NLTK library. 

Our next task was to reduce our set of tokens into common and similar words. Initially, we only attempted to correct the spelling of the token, without any further processing. However, we were left with a lot of noise and useless/mispelled words that doesn't help our prediction. We then improved our algorithm by first checking to see if each token exist in pre-defined dictionary (en-US). This means that our resulting set of words should only contain correct words. We then lowercase them, and retrieve their "stem" using NLTK's LancasterStemmer package, to make sure that past tense and plurals are reduced to its stem word. At the end of this process, we ended up with a much smaller set of features.

Once we were able to get a list of all unique words that exists in every reviews, we use that list to crossreference with every review and count the word's frequency.

In [ ]:
def findwordsandcount(df, review_count):
    for index, row in df.T.iteritems():
        # keep only alphabetical letters and space
        new_string = re.sub(r"[^a-zA-Z\ ]+"," ", row["text"])
        # parse entire review text into tokens
        tokens = nltk.word_tokenize(new_string)
        # keep only reviews that has more than review_count # of words
        df.at[index,'count'] = len(tokens) 
        if len(tokens) >=review_count:
            # for reviews that we want to keep, we apply the removetoken function
            df.at[index,'text'] = removetoken(tokens)

In [ ]:
def removetoken(tokens):
    save_token =[]
    for atoken in tokens:
        if (len(atoken) != 1) | (atoken == 'a') | (atoken == 'i') \
        | (atoken == 'A')| (atoken == 'I'):
            # we check to see if it's in our dictionary
            if (dictionary.check(atoken) == False):
                continue
            # we lowercase it to make parsing consistent
            atoken = str.lower(atoken)
            # we get the root word of each token, to make parsing consistent
            atoken = st.stem(atoken)
            save_token.append(atoken)
    return save_token

In [ ]:
def finduniqueword(df):
    all_reiew = []
    for index, row in df.T.iteritems():
        all_reiew  = all_reiew + df.at[index,'text']
        all_reiew = list(set(all_reiew))
    return all_reiew

In [ ]:
def countwords(df,unique_word):
    dataset_df = pd.DataFrame(columns=['review_id','stars'] + unique_word)

    # we count how many times each word appears in each review
    for index, a_review in df.iterrows():
        empty_row = dict(zip(['review_id','stars'] +unique_word , [0]*(len(unique_word)+2)) )
        counts  = Counter(a_review.text)

        empty_row['review_id'] = a_review.review_id
        empty_row['stars'] = a_review.stars
        empty_row.update(counts)
        dataset_df.loc[index] = empty_row
    return dataset_df

In [ ]:
words_list = []
cols = ['review_id','stars','text']

all_dataset_df = pd.DataFrame(columns=['review_id','stars'])

# record the time
start_time = time.time()
clean_time = time.time()
chunk_count = 0

for part_df in pd.read_json(outputname, lines=True, chunksize=chunk_size):
    part_df = part_df[cols]
    part_df['count']  = 0
    findwordsandcount(part_df, min_keep_size)
    print ("Chunk: ", chunk_count)
    print("findwordsandcount: ", (time.time() - start_time))
    start_time = time.time()
    
    part_df = part_df.loc[part_df['count'] >= min_keep_size]
    unique_word = finduniqueword(part_df)
    unique_word.sort()
    
    print("finduniqueword: ", (time.time() - start_time))
    start_time = time.time()

    dataset_df = countwords(part_df,unique_word)
    
    all_dataset_df = all_dataset_df.append(dataset_df,sort=False)
    print("countwords: ", (time.time() - start_time))
    start_time = time.time()
    chunk_count+=1

print("Cleaning time: ", (time.time() - clean_time))

Chunk:  0
findwordsandcount:  0.7892367839813232
finduniqueword:  0.010324954986572266
countwords:  7.64995002746582
Chunk:  1
findwordsandcount:  0.6903131008148193
finduniqueword:  0.00862574577331543
countwords:  7.050591945648193
Chunk:  2
findwordsandcount:  0.6202528476715088
finduniqueword:  0.006911039352416992
countwords:  5.700318098068237
Chunk:  3
findwordsandcount:  0.6181550025939941
finduniqueword:  0.006780862808227539
countwords:  5.908515930175781
Chunk:  4
findwordsandcount:  0.6343197822570801
finduniqueword:  0.011677026748657227
countwords:  7.1894121170043945
Chunk:  5
findwordsandcount:  0.5121910572052002
finduniqueword:  0.009585142135620117
countwords:  5.58926796913147
Chunk:  6
findwordsandcount:  0.6115961074829102
finduniqueword:  0.00706791877746582
countwords:  6.22661566734314
Chunk:  7
findwordsandcount:  0.6816027164459229
finduniqueword:  0.008348941802978516
countwords:  7.212285041809082
Chunk:  8
findwordsandcount:  0.5819821357727051
finduniquew

In [ ]:
# filling NaN with 0s
all_dataset_df = all_dataset_df.fillna(0)
all_dataset_df

### Saving the cleaned data

In [ ]:
all_dataset_df.to_csv(download_name, index=False)

## 3. Train the model and predict dataset without using PCA

In [ ]:
all_dataset_df = pd.read_csv(download_name)
columns_word_only = all_dataset_df.columns.drop(['stars','review_id'])

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(\
    all_dataset_df[columns_word_only], \
    all_dataset_df["stars"], test_size=0.2,random_state=1)

In [ ]:
for k_value in range(4,20,2):
    knn_model = KNeighborsClassifier(n_neighbors=k_value)
    knn_model.fit(x_train, y_train)
    knn_y_predict = knn_model.predict(x_test)
    print("K value: ", k_value, "   Accuracy: ", accuracy_score(y_test, knn_y_predict))

### Confusion Matrix

knn_model = KNeighborsClassifier(n_neighbors=14)
knn_model.fit(x_train, y_train)
knn_y_predict = knn_model.predict(x_test)
print("K nearest neighbor without PCA")
print(confusion_matrix(y_test, knn_y_predict))

## 4. Train the model and predict dataset using PCA

In [ ]:
columns_word_only = all_dataset_df.columns.drop(['stars','review_id'])

pca = decomposition.PCA(n_components= pca_components)
principalComponents = pca.fit_transform(all_dataset_df[columns_word_only])
pca_x_Df = pd.DataFrame(data = principalComponents)

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(\
    pca_x_Df, all_dataset_df["stars"], test_size=0.2,random_state=1)

### K nearest neighbor (KNeighborsClassifier)

In [ ]:
for k_value in range(4,20,2):
    knn_model = KNeighborsClassifier(n_neighbors=k_value)
    knn_model.fit(x_train, y_train)
    knn_y_predict = knn_model.predict(x_test)
    print("K value: ", k_value, "   Accuracy: ", accuracy_score(y_test, knn_y_predict))

### Confusion Matrix

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=8)
knn_model.fit(x_train, y_train)
knn_y_predict = knn_model.predict(x_test)
print("K nearest neighbor(with PCA)")
print(confusion_matrix(y_test, knn_y_predict))

## 5. Analyze & Conclude the results

When we use K-NearestNeighbor model, the highest accuracy we obtained was when K = 8, for both with PCA and without PCA. This means that we can reliably use a smaller subset of features, instead of having to use all of the features, reducing our run time by a substantial amount. 

Because there were 6 labels (ratings), the random select probability for correct prediction is 1/6. Using our algorithm, we were able to make a prediction with an accuracy of approximately 2/5. Although it is not entirely accurate, it is a noticable mark in improvement on random selection. We predict that if we were to use the entire 6 million reviews, our accuracy will improve.